In [1]:
# USAGE
# python save_model.py --dataset malaria --model saved_model.model

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from pyimagesearch.resnet import ResNet
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

Using TensorFlow backend.


In [2]:
# initialize the number of training epochs and batch size
NUM_EPOCHS = 25
BS = 32

In [3]:
#derive the path to the directories containing the training,
# validation, and testing splits, respectively
TRAIN_PATH = "/home/user/Desktop/keras-save-load/malaria/training"
VAL_PATH = "/home/user/Desktop/keras-save-load/malaria/validation"
TEST_PATH = "C:/Users/User/Desktop/keras-save-load/malaria/testing/"

In [4]:
# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

In [5]:
print(list(paths.list_images(TRAIN_PATH)),totalTrain,totalVal,totalTest)

[] 0 0 100


In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
rescale=1 / 255.0,
rotation_range=20,
zoom_range=0.05,
width_shift_range=0.05,
height_shift_range=0.05,
shear_range=0.05,
horizontal_flip=True,
fill_mode="nearest")

In [ ]:
# initialize the validation (and testing) data augmentation object
valAug = ImageDataGenerator(rescale=1 / 255.0)

In [ ]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
TRAIN_PATH,
class_mode="categorical",
target_size=(64, 64),
color_mode="rgb",
shuffle=True,
batch_size=32)

In [ ]:
# initialize the validation generator
valGen = valAug.flow_from_directory(
VAL_PATH,
class_mode="categorical",
target_size=(64, 64),
color_mode="rgb",
shuffle=False,
batch_size=BS)

In [ ]:
# initialize the testing generator
testGen = valAug.flow_from_directory(
TEST_PATH,
class_mode="categorical",
target_size=(64, 64),
color_mode="rgb",
shuffle=False,
batch_size=BS)

In [ ]:
# initialize our Keras implementation of ResNet model and compile it
model = ResNet.build(64, 64, 3, 2, (2, 2, 3),
(32, 64, 128, 256), reg=0.0005)
opt = SGD(lr=1e-1, momentum=0.9, decay=1e-1 / NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
metrics=["accuracy"])

In [ ]:
# train our Keras model
H = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // BS,
    validation_data=valGen,
    validation_steps=totalVal // BS,
    epochs=NUM_EPOCHS)

In [ ]:
# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,steps=(totalTest // BS) + 1)

In [ ]:
print(predIdxs,predIdxs.shape)

In [ ]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)


In [ ]:
print(predIdxs)
print(testGen.classes)
target_names=testGen.class_indices.keys()
print(target_names)

In [ ]:
# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,target_names=testGen.class_indices.keys()))

In [ ]:
# save the network to disk
print("[INFO] serializing network to '{}'...".format("saved model"))
model.save("saved model")

In [ ]:
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
#plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
#plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('behnoud1.png')